### 여러 페이지에서 가져온 배우 정보를 MongoDB 에 저장하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [2]:
'''
MongoDB 연결하기
'''

conn = pymongo.MongoClient()
cine = conn.cine
collection_cine = cine.collection_cine


In [3]:
'''
collection 삭제
'''
# collection_cine.drop()

'\ncollection 삭제\n'

In [4]:
actors_info_list = []

In [5]:
url = "http://www.cine21.com/rank/person/content"

post_data = {}
post_data["section"] = "actor"
post_data["period_start"] = "2020-12"
post_data["gender"] = "all"


In [6]:
# 1 page ~ 20 page

for idx in range(1, 51):
    post_data["page"] = idx
    
    resp = requests.post(url, post_data)
    soup = BeautifulSoup(resp.content, "html.parser")
    
    actors = soup.select("li.people_li div.name")
    hits   = soup.select("ul.num_info > li > strong")
    movies = soup.select("ul.mov_list")
    rankings = soup.select("li.people_li > span.grade")
    
    for idx, actor in enumerate(actors):
        actor_name = (actor.text).split("(")[0]
        actor_hits = int(hits[idx].text.replace(",",""))
        movie_titles = movies[idx].select("li a span")
        
        movie_title_list = []
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        
        actors_info_dict = {}
        actors_info_dict["배우이름"] = actor_name
        actors_info_dict["흥행지수"] = actor_hits
        actors_info_dict["출연영화"] = movie_title_list
        actors_info_dict["랭킹"] = rankings[idx].text
        
        # 배우 상세 정보 page 로 이동하기 위해서 
        # url 저장하기
        actor_link = "http://www.cine21.com" + actor.select_one("a").attrs["href"]
        actor_resp = requests.get(actor_link)
        
        actor_soup = BeautifulSoup(actor_resp.content, "html.parser")
        
        default_info = actor_soup.select_one("ul.default_info")
        actor_details = default_info.select("li")
        
        for actor_item in actor_details:
            actor_item_key = actor_item.select_one("span.tit").text
            actor_item_value = re.sub("<span.*?>.*?</span>", "", str(actor_item))
            actor_item_value = re.sub("<.*?>", "", actor_item_value)
            
            actors_info_dict[actor_item_key] = actor_item_value
            
    actors_info_list.append(actors_info_dict)
        
actors_info_list        

[{'배우이름': '변요한',
  '흥행지수': 17851,
  '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
  '랭킹': '7',
  '직업': '배우',
  '생년월일': '1986-04-29',
  '성별': '남',
  '홈페이지': '\nhttps://www.facebook.com/saram.byunyohan\nhttps://www.instagram.com/qusdygkssla/\n'},
 {'배우이름': '음문석',
  '흥행지수': 15263,
  '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'],
  '랭킹': '14',
  '다른 이름': 'sic',
  '직업': '배우',
  '생년월일': '1982-12-07',
  '성별': '남'},
 {'배우이름': '전혜진',
  '흥행지수': 12619,
  '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
  '랭킹': '21',
  '다른 이름': '전이다',
  '직업': '배우',
  '생년월일': '1976-08-10',
  '성별': '여',
  '신장/체중': '171cm, 49kg',
  '학교': '상명대학교 영화과',
  '취미': '영화감상, 노래 부르기',
  '특기': '수영',
  '소속사': '호두엔터테인먼트'},
 {'배우이름': '허성태',
  '흥행지수': 10308,
  '출연영화': ['헌트', '스텔라', '블랙머니', '말모이'],
  '랭킹': '28',
  '직업': '배우',
  '생년월일': '1977-10-20',
  '성별': '남'},
 {'배우이름': '정만식',
  '흥행지수': 8779,
  '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다'],
  '랭킹': '35',
  '직업': '배우',
  '생

In [7]:
# collection_cine.delete_many({})

In [8]:
collection_cine.insert_many(actors_info_list)

In [9]:
collection_cine.count_documents({})

50

### 데이터 분석 준비하기
#### 1) key(field)명 변경하기
- MongoDB 에 저장되어 있는 document 의 key(field) 이름 변경하기
- update_one() / update_many()

In [10]:
collection_cine.find_one({})

{'_id': ObjectId('6322e0a859013109670e4c40'),
 '배우이름': '변요한',
 '흥행지수': 17851,
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '랭킹': '7',
 '직업': '배우',
 '생년월일': '1986-04-29',
 '성별': '남',
 '홈페이지': '\nhttps://www.facebook.com/saram.byunyohan\nhttps://www.instagram.com/qusdygkssla/\n'}

In [11]:
import pprint

documents = collection_cine.find({}).limit(3)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'_id': ObjectId('6322e0a859013109670e4c40'),
 '랭킹': '7',
 '배우이름': '변요한',
 '생년월일': '1986-04-29',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '홈페이지': '\n'
         'https://www.facebook.com/saram.byunyohan\n'
         'https://www.instagram.com/qusdygkssla/\n',
 '흥행지수': 17851}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c41'),
 '다른 이름': 'sic',
 '랭킹': '14',
 '배우이름': '음문석',
 '생년월일': '1982-12-07',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'],
 '흥행지수': 15263}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c42'),
 '다른 이름': '전이다',
 '랭킹': '21',
 '배우이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행지수'

In [12]:
'''
배우이름 --> 배우 이름  : $rename
'''
collection_cine.update_many({}, {"$rename" : {"배우이름":"배우 이름"}})

In [13]:
documents = collection_cine.find({}).limit(3)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c40'),
 '랭킹': '7',
 '배우 이름': '변요한',
 '생년월일': '1986-04-29',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '홈페이지': '\n'
         'https://www.facebook.com/saram.byunyohan\n'
         'https://www.instagram.com/qusdygkssla/\n',
 '흥행지수': 17851}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c41'),
 '다른 이름': 'sic',
 '랭킹': '14',
 '배우 이름': '음문석',
 '생년월일': '1982-12-07',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'],
 '흥행지수': 15263}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c42'),
 '다른 이름': '전이다',
 '랭킹': '21',
 '배우 이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행

### 조회할 때 정렬하기(오름차순 / 내림차순)
- sort() 메소드
- 오름차순 : pymongo.ASCENDING (기본값)
- 내림차순 : pymongo.DESCENDING

In [14]:
documents = collection_cine.find({}).sort("생년월일").limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c53'),
 '랭킹': '140',
 '배우 이름': '염정아',
 '성별': '여',
 '소속사': '아티스트컴퍼니',
 '직업': '배우',
 '출연영화': ['외계+인 1부', '장화, 홍련', '인생은 아름다워', '시동', '강철비2: 정상회담', '미성년'],
 '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/yum-jung-ah/\n',
 '흥행지수': 1349}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c54'),
 '랭킹': '147',
 '배우 이름': '이유영',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['장르만 로맨스', '간호중', '디바', '원더풀 고스트'],
 '흥행지수': 1176}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c5d'),
 '랭킹': '210',
 '배우 이름': '정재헌',
 '성별': '남',
 '직업': '성우',
 '출연영화': ['바다 탐험대 옥토넛: 탐험선 대작전',
          '극장판 바다 탐험대 옥토넛 : 해저동굴 대탈출',
          '바다 탐험대 옥토넛 : 육지수호 대작전',
          '극장판 바다 탐험대 옥토넛: 대산호초 보호작전',
          '극장판 바다 탐험대 옥토넛: 불의 고리 대폭발',
          '바다 탐험대 옥토넛 시즌4: 바다 괴물 대소동'],
 '흥행지수': 501}
----------------------------------------------------

In [15]:
documents = collection_cine.find({}).sort("생년월일", pymongo.ASCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c53'),
 '랭킹': '140',
 '배우 이름': '염정아',
 '성별': '여',
 '소속사': '아티스트컴퍼니',
 '직업': '배우',
 '출연영화': ['외계+인 1부', '장화, 홍련', '인생은 아름다워', '시동', '강철비2: 정상회담', '미성년'],
 '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/yum-jung-ah/\n',
 '흥행지수': 1349}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c54'),
 '랭킹': '147',
 '배우 이름': '이유영',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['장르만 로맨스', '간호중', '디바', '원더풀 고스트'],
 '흥행지수': 1176}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c5d'),
 '랭킹': '210',
 '배우 이름': '정재헌',
 '성별': '남',
 '직업': '성우',
 '출연영화': ['바다 탐험대 옥토넛: 탐험선 대작전',
          '극장판 바다 탐험대 옥토넛 : 해저동굴 대탈출',
          '바다 탐험대 옥토넛 : 육지수호 대작전',
          '극장판 바다 탐험대 옥토넛: 대산호초 보호작전',
          '극장판 바다 탐험대 옥토넛: 불의 고리 대폭발',
          '바다 탐험대 옥토넛 시즌4: 바다 괴물 대소동'],
 '흥행지수': 501}
----------------------------------------------------

In [16]:
documents = collection_cine.find({}).sort("생년월일", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c6e'),
 '랭킹': '329',
 '배우 이름': '이레',
 '생년월일': '2006-03-12',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['반도', '너의 이름은.', '증인', '개를 훔치는 완벽한 방법'],
 '홈페이지': '\nhttps://www.facebook.com/leere.vv/\n',
 '흥행지수': 154}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c4f'),
 '랭킹': '112',
 '배우 이름': '이재인',
 '생년월일': '2004-02-06',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['발신제한', '아이 캔 스피크', '봉오동 전투', '어른도감'],
 '흥행지수': 2027}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c46'),
 '랭킹': '49',
 '배우 이름': '성유빈',
 '생년월일': '2000-07-25',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['마녀(魔女) Part2. The Other One',
          '장르만 로맨스',
          '니 부모 얼굴이 보고 싶다',
          '아이 캔 스피크',
          '봉오동 전투',
          '윤희에게'],
 '홈페이지': '\n'
         'https://www.facebook.com/%EC%84%B1%EC%9C%A0%EB%B9%88-1650644645187320/?fref=ts\n',
 '흥행지수': 6257}
------------

In [17]:
'''
"특기"라는 key 값이 있는 document 만 조회하기
                  ㄴ $exists
'''
documents = collection_cine.find({"특기" : {"$exists" : True}})

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c42'),
 '다른 이름': '전이다',
 '랭킹': '21',
 '배우 이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행지수': 12619}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c45'),
 '다른 이름': 'Byung-hun Lee;BH Lee',
 '랭킹': '42',
 '배우 이름': '이병헌',
 '생년월일': '1970-07-12',
 '성별': '남',
 '신장/체중': '177cm, 72kg',
 '직업': '배우',
 '출연영화': ['비상선언', '번지점프를 하다', '공동경비구역 JSA', '그것만이 내 세상'],
 '취미': '모자수집, 여행',
 '특기': '태권도, 스노우보드, 수영, 팔씨름',
 '학교': '한양대학교 불어문학과',
 '홈페이지': '\nhttp://www.leebyunghun.kr/\n',
 '흥행지수': 7467}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c55'),
 '랭킹': '154',
 '배우 이름': '배종옥',
 '생년월일': '1964-05-13',
 '성별': '여',
 '신장/체중': '163cm, 45kg',
 '직업': '배우',


In [18]:
'''
"특기" 라는 key 값이 있는 배우들만 
"흥행지수" 순으로 정렬하기 (오름차순)
'''
documents = collection_cine.find({"특기" : {"$exists" : True}}).sort("흥행지수")

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c6a'),
 '다른 이름': '최수영; 소녀시대; girlsgeneration; girls generation',
 '랭킹': '301',
 '배우 이름': '수영',
 '생년월일': '1990-02-10',
 '성별': '여',
 '소속사': 'SM엔터테인먼트',
 '신장/체중': '170cm',
 '직업': '가수',
 '출연영화': ['막다른 골목의 추억', '새해전야'],
 '특기': '일본어',
 '홈페이지': '\nhttps://www.instagram.com/hotsootuff/\n',
 '흥행지수': 191}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c63'),
 '랭킹': '252',
 '배우 이름': '이연희',
 '생년월일': '1988-01-09',
 '성별': '여',
 '소속사': 'VAST엔터테인먼트',
 '신장/체중': '170cm, 46kg',
 '직업': '배우',
 '출연영화': ['새해전야'],
 '취미': '영화감상, 독서',
 '특기': '수영',
 '학교': '중앙대학교 연극영화학',
 '홈페이지': '\nhttp://www.vastenm.com/\n',
 '흥행지수': 318}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c60'),
 '랭킹': '231',
 '배우 이름': '류현경',
 '생년월일': '1983-03-10',
 '성별': '여',
 '신장/체중': '162cm',
 '직업': '배우',
 '출연영화': ['아이', '장르만 로맨스', '기도하는 남자', '태양은 없다'],
 '취미': '영화

In [19]:
'''
"특기" 라는 key 값이 있는 배우들만 
"흥행지수" 내림차순으로 정렬하기 - 10 개만
'''
documents = \
  collection_cine.find({"특기" : {"$exists" : True}})\
                 .sort("흥행지수", pymongo.DESCENDING)\
                 .limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c42'),
 '다른 이름': '전이다',
 '랭킹': '21',
 '배우 이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행지수': 12619}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c45'),
 '다른 이름': 'Byung-hun Lee;BH Lee',
 '랭킹': '42',
 '배우 이름': '이병헌',
 '생년월일': '1970-07-12',
 '성별': '남',
 '신장/체중': '177cm, 72kg',
 '직업': '배우',
 '출연영화': ['비상선언', '번지점프를 하다', '공동경비구역 JSA', '그것만이 내 세상'],
 '취미': '모자수집, 여행',
 '특기': '태권도, 스노우보드, 수영, 팔씨름',
 '학교': '한양대학교 불어문학과',
 '홈페이지': '\nhttp://www.leebyunghun.kr/\n',
 '흥행지수': 7467}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c55'),
 '랭킹': '154',
 '배우 이름': '배종옥',
 '생년월일': '1964-05-13',
 '성별': '여',
 '신장/체중': '163cm, 45kg',
 '직업': '배우',


In [20]:
'''
생년월일 이라는 key 값이 있는 배우들만
흥행지수 내림차순으로 정렬하기 - 5 개만
'''
documents = \
  collection_cine.find({"생년월일" : {"$exists" : True}})\
                 .sort("흥행지수", pymongo.DESCENDING)\
                 .limit(5)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('6322e0a859013109670e4c40'),
 '랭킹': '7',
 '배우 이름': '변요한',
 '생년월일': '1986-04-29',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '홈페이지': '\n'
         'https://www.facebook.com/saram.byunyohan\n'
         'https://www.instagram.com/qusdygkssla/\n',
 '흥행지수': 17851}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c41'),
 '다른 이름': 'sic',
 '랭킹': '14',
 '배우 이름': '음문석',
 '생년월일': '1982-12-07',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'],
 '흥행지수': 15263}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c42'),
 '다른 이름': '전이다',
 '랭킹': '21',
 '배우 이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행

In [21]:
'''
생년월일 key 값이 있는 배우들만
흥행지수 순서로 정렬해서 출력하기
내림차순, 5 명만 
배우 이름 key 만 출력하세요
'''

documents = \
  collection_cine.find({"생년월일" : {"$exists":True}}, {"배우 이름":1, "흥행지수":1,"_id":0})\
                 .sort("흥행지수").limit(10)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '박지윤', '흥행지수': 123}
--------------------------------------------------------------------------------
{'배우 이름': '진서연', '흥행지수': 133}
--------------------------------------------------------------------------------
{'배우 이름': '정희태', '흥행지수': 145}
--------------------------------------------------------------------------------
{'배우 이름': '이레', '흥행지수': 154}
--------------------------------------------------------------------------------
{'배우 이름': '이지현', '흥행지수': 159}
--------------------------------------------------------------------------------
{'배우 이름': '서강준', '흥행지수': 174}
--------------------------------------------------------------------------------
{'배우 이름': '수영', '흥행지수': 191}
--------------------------------------------------------------------------------
{'배우 이름': '김세준', '흥행지수': 195}
--------------------------------------------------------------------------------
{'배우 이름': '김도윤', '흥행지수': 211}
--------------------------------------------------------------------------------
{'배

In [22]:
'''
내림차순 : pymongo.DESCENDING
'''
documents = \
  collection_cine.find({"생년월일" : {"$exists":True}}, {"배우 이름":1, "흥행지수":1,"_id":0})\
                 .sort("흥행지수", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '변요한', '흥행지수': 17851}
--------------------------------------------------------------------------------
{'배우 이름': '음문석', '흥행지수': 15263}
--------------------------------------------------------------------------------
{'배우 이름': '전혜진', '흥행지수': 12619}
--------------------------------------------------------------------------------
{'배우 이름': '허성태', '흥행지수': 10308}
--------------------------------------------------------------------------------
{'배우 이름': '정만식', '흥행지수': 8779}
--------------------------------------------------------------------------------
{'배우 이름': '이병헌', '흥행지수': 7467}
--------------------------------------------------------------------------------
{'배우 이름': '성유빈', '흥행지수': 6257}
--------------------------------------------------------------------------------
{'배우 이름': '한예리', '흥행지수': 5315}
--------------------------------------------------------------------------------
{'배우 이름': '박정민', '흥행지수': 4722}
---------------------------------------------------------------------

In [23]:
'''
흥행지수가 10000 이상인 배우만 조회하기
'''
documents = collection_cine.find({"흥행지수":{"$gte":10000}})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'_id': ObjectId('6322e0a859013109670e4c40'),
 '랭킹': '7',
 '배우 이름': '변요한',
 '생년월일': '1986-04-29',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '홈페이지': '\n'
         'https://www.facebook.com/saram.byunyohan\n'
         'https://www.instagram.com/qusdygkssla/\n',
 '흥행지수': 17851}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c41'),
 '다른 이름': 'sic',
 '랭킹': '14',
 '배우 이름': '음문석',
 '생년월일': '1982-12-07',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'],
 '흥행지수': 15263}
--------------------------------------------------------------------------------
{'_id': ObjectId('6322e0a859013109670e4c42'),
 '다른 이름': '전이다',
 '랭킹': '21',
 '배우 이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행

In [24]:
'''
흥행지수가 10000 이상인 배우만 조회하기
흥행지수, 배우 이름만 출력하기
흥행지수 내림차순으로 정렬하기
'''
documents = \
  collection_cine.find({"흥행지수":{"$gte":5000}}, \
                       {"흥행지수":1,"배우 이름":1, "출연영화":1,"_id":0})\
                 .sort("흥행지수", pymongo.DESCENDING)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '변요한',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '흥행지수': 17851}
--------------------------------------------------------------------------------
{'배우 이름': '음문석', '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'], '흥행지수': 15263}
--------------------------------------------------------------------------------
{'배우 이름': '전혜진',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '흥행지수': 12619}
--------------------------------------------------------------------------------
{'배우 이름': '허성태', '출연영화': ['헌트', '스텔라', '블랙머니', '말모이'], '흥행지수': 10308}
--------------------------------------------------------------------------------
{'배우 이름': '정만식',
 '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다'],
 '흥행지수': 8779}
--------------------------------------------------------------------------------
{'배우 이름': '이병헌',
 '출연영화': ['비상선언', '번지점프를 하다', '공동경비구역 JSA', '그것만이 내 세상'],
 '흥행지수': 7467}
------------------------------------------------------

In [25]:
'''
흥행지수가 10000 이상인 배우 중에서
"모가디슈" 라는 영화에 출연한 사람만 조회하기
배우 이름, 흥행지수, 출연영화 만 조회(출력)하기
'''
documents = \
  collection_cine.find({"흥행지수":{"$gte":10000}, "출연영화":"비상선언"},\
                       {"배우 이름":1, "흥행지수":1, "출연영화":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [26]:
'''
흥행지수가 5000 이 넘으면서
"비상선언" 또는 "이웃사촌" 에 출연한 배우들을 조회하세요
배우 이름, 출연영화 키만 출력하세요
'''
documents = \
  collection_cine.find({"흥행지수":{"$gte":5000}, "출연영화":"비상선언"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})

documents = \
  collection_cine.find({"흥행지수":{"$gte":5000}, "출연영화":"이웃사촌"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})

### find() - or

In [27]:
documents = \
  collection_cine.find({"흥행지수":{"$gte":5000}, \
                        "$or" : [{"출연영화":"비상선언"}, {"출연영화":"이웃사촌"}]},\
                       {"배우 이름":1, "출연영화":1, "흥행지수":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '이병헌',
 '출연영화': ['비상선언', '번지점프를 하다', '공동경비구역 JSA', '그것만이 내 세상'],
 '흥행지수': 7467}
--------------------------------------------------------------------------------


### find() - and

In [28]:
documents = \
  collection_cine.find({"흥행지수":{"$gte":5000}, \
                        "$and" : [{"출연영화":"비상선언"}, {"출연영화":"그것만이 내 세상"}]},\
                       {"배우 이름":1, "출연영화":1, "흥행지수":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '이병헌',
 '출연영화': ['비상선언', '번지점프를 하다', '공동경비구역 JSA', '그것만이 내 세상'],
 '흥행지수': 7467}
--------------------------------------------------------------------------------


### find() - nor (not or)

In [29]:
documents = \
  collection_cine.find({"$nor":[{"흥행지수": {"$gte":10000}}, {"흥행지수":{"$lte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '정만식', '흥행지수': 8779}
--------------------------------------------------------------------------------
{'배우 이름': '이병헌', '흥행지수': 7467}
--------------------------------------------------------------------------------
{'배우 이름': '성유빈', '흥행지수': 6257}
--------------------------------------------------------------------------------
{'배우 이름': '한예리', '흥행지수': 5315}
--------------------------------------------------------------------------------
{'배우 이름': '박정민', '흥행지수': 4722}
--------------------------------------------------------------------------------
{'배우 이름': '천우희', '흥행지수': 4084}
--------------------------------------------------------------------------------
{'배우 이름': '류경수', '흥행지수': 3790}
--------------------------------------------------------------------------------
{'배우 이름': '진경', '흥행지수': 3043}
--------------------------------------------------------------------------------
{'배우 이름': '김신영', '흥행지수': 2841}
--------------------------------------------------------------------------

In [30]:
documents = \
  collection_cine.find({"$and":[{"흥행지수": {"$lte":10000}}, {"흥행지수":{"$gte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '정만식', '흥행지수': 8779}
--------------------------------------------------------------------------------
{'배우 이름': '이병헌', '흥행지수': 7467}
--------------------------------------------------------------------------------
{'배우 이름': '성유빈', '흥행지수': 6257}
--------------------------------------------------------------------------------
{'배우 이름': '한예리', '흥행지수': 5315}
--------------------------------------------------------------------------------
{'배우 이름': '박정민', '흥행지수': 4722}
--------------------------------------------------------------------------------
{'배우 이름': '천우희', '흥행지수': 4084}
--------------------------------------------------------------------------------
{'배우 이름': '류경수', '흥행지수': 3790}
--------------------------------------------------------------------------------
{'배우 이름': '진경', '흥행지수': 3043}
--------------------------------------------------------------------------------
{'배우 이름': '김신영', '흥행지수': 2841}
--------------------------------------------------------------------------

In [31]:
documents = \
  collection_cine.find({"$or":[{"흥행지수": {"$gte":10000}}, {"흥행지수":{"$lte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '변요한', '흥행지수': 17851}
--------------------------------------------------------------------------------
{'배우 이름': '음문석', '흥행지수': 15263}
--------------------------------------------------------------------------------
{'배우 이름': '전혜진', '흥행지수': 12619}
--------------------------------------------------------------------------------
{'배우 이름': '허성태', '흥행지수': 10308}
--------------------------------------------------------------------------------
{'배우 이름': '이주영', '흥행지수': 898}
--------------------------------------------------------------------------------
{'배우 이름': '임성재', '흥행지수': 760}
--------------------------------------------------------------------------------
{'배우 이름': '유오성', '흥행지수': 681}
--------------------------------------------------------------------------------
{'배우 이름': '이혜영', '흥행지수': 597}
--------------------------------------------------------------------------------
{'배우 이름': '강희선', '흥행지수': 551}
--------------------------------------------------------------------------

### find() - in(포함되어 있음), nin(not in)(포함되어 있지 않음)

In [32]:
'''
흥행지수가 12619, 15263 인 배우만 조회하기 - $in 
배우 이름, 흥행지수 만 조회하기
'''
documents = collection_cine.find({"흥행지수":{"$in":[12619, 15263]}},\
                                 {"배우 이름":1, "흥행지수":1, "_id":0})
                                  
for document in documents:
    pprint.pprint(document)    

{'배우 이름': '음문석', '흥행지수': 15263}
{'배우 이름': '전혜진', '흥행지수': 12619}


In [33]:
'''
흥행지수가 12619, 15263 이 아닌 배우만 조회하기 -  $nin
배우 이름, 흥행지수만 조회하기
'''
documents = collection_cine.find({"흥행지수":{"$nin":[12619, 15263]}},\
                                 {"배우 이름":1, "흥행지수":1, "_id":0})
for document in documents:
    pprint.pprint(document)    

{'배우 이름': '변요한', '흥행지수': 17851}
{'배우 이름': '허성태', '흥행지수': 10308}
{'배우 이름': '정만식', '흥행지수': 8779}
{'배우 이름': '이병헌', '흥행지수': 7467}
{'배우 이름': '성유빈', '흥행지수': 6257}
{'배우 이름': '한예리', '흥행지수': 5315}
{'배우 이름': '박정민', '흥행지수': 4722}
{'배우 이름': '천우희', '흥행지수': 4084}
{'배우 이름': '류경수', '흥행지수': 3790}
{'배우 이름': '진경', '흥행지수': 3043}
{'배우 이름': '김신영', '흥행지수': 2841}
{'배우 이름': '한지민', '흥행지수': 2558}
{'배우 이름': '채수빈', '흥행지수': 2263}
{'배우 이름': '이재인', '흥행지수': 2027}
{'배우 이름': '김희원', '흥행지수': 1753}
{'배우 이름': '이선균', '흥행지수': 1684}
{'배우 이름': '우정신', '흥행지수': 1540}
{'배우 이름': '염정아', '흥행지수': 1349}
{'배우 이름': '이유영', '흥행지수': 1176}
{'배우 이름': '배종옥', '흥행지수': 1071}
{'배우 이름': '이선', '흥행지수': 1012}
{'배우 이름': '이주영', '흥행지수': 898}
{'배우 이름': '임성재', '흥행지수': 760}
{'배우 이름': '유오성', '흥행지수': 681}
{'배우 이름': '이혜영', '흥행지수': 597}
{'배우 이름': '강희선', '흥행지수': 551}
{'배우 이름': '이규형', '흥행지수': 528}
{'배우 이름': '정재헌', '흥행지수': 501}
{'배우 이름': '이홍내', '흥행지수': 460}
{'배우 이름': '진기주', '흥행지수': 419}
{'배우 이름': '류현경', '흥행지수': 399}
{'배우 이름': '이수혁', '흥행지수': 363}
{'배우 이름': '브루스 칸', 

In [34]:
'''
흥행지수가 12619, 15263 이 아니고(아니면서)
10000 이상인 배우 이름과 흥행지수를 조회하세요
  ㄴ 10000 미만이 아닌
[힌트]
  $nor 와 $in 같이 사용하기
  
1)   흥행지수가 12619, 15263 인 조건

     {"흥행지수":{"$in":[12619, 15263]}
     
     
2)   흥행지수가 10000 미만인 조건

     {"흥행지수":{"$lt":10000}
  
3)   {"$nor" : [{"흥행지수":{"$in":[12619, 15263]}, {"흥행지수":{"$lt":10000}]}
'''
documents = \
  collection_cine.find({"$nor" : [{"흥행지수":{"$in":[12619, 15263]}}, \
                                  {"흥행지수":{"$lt":10000}}]},\
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    

{'배우 이름': '변요한', '흥행지수': 17851}
{'배우 이름': '허성태', '흥행지수': 10308}


### find() - skip() / limit()
- skip(n) - 시작 부분에서 n 개 건너뜀 (n+1 부터 검색함)
- limit(n) - n 개만 조회함

In [35]:
'''
흥행지수가 10000 을 초과하는 배우들의
배우 이름, 흥행지수를 조회하세요
'''
documents = \
  collection_cine.find({"흥행지수":{"$gt":10000}},\
                       {"흥행지수":1, "배우 이름":1, "출연영화":1,"_id":0})

for document in documents:
    pprint.pprint(document)
print('-' * 80)    
  

{'배우 이름': '변요한',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '흥행지수': 17851}
{'배우 이름': '음문석', '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'], '흥행지수': 15263}
{'배우 이름': '전혜진',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '흥행지수': 12619}
{'배우 이름': '허성태', '출연영화': ['헌트', '스텔라', '블랙머니', '말모이'], '흥행지수': 10308}
--------------------------------------------------------------------------------


In [36]:
'''
모가디슈에 출연한 배우들의 배우 이름, 출연영화를 출력하세요
'''
documents = \
  collection_cine.find({"출연영화":"모가디슈"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)

{'배우 이름': '정만식',
 '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다']}


In [37]:
'''
모가디슈나 타짜에 출연한 배우들의 
배우 이름, 출연영화를 조회하세요
'''
documents = \
  collection_cine.find({"$or":[{"출연영화":"모가디슈"},{"출연영화":"타짜"}]},\
                        {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)
    

{'배우 이름': '정만식',
 '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다']}


In [38]:
'''
모가디슈와 타짜에 출연한 배우들의 
배우 이름, 출연영화를 조회하세요
'''
documents =\
  collection_cine.find({"출연영화":{"$all":["모가다슈", "헌트"]}},\
                       {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)

In [39]:
documents = collection_cine.find({},{"_id":0})
# print(len(list(documents)))
for document in documents:
    pprint.pprint(document)

{'랭킹': '7',
 '배우 이름': '변요한',
 '생년월일': '1986-04-29',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '홈페이지': '\n'
         'https://www.facebook.com/saram.byunyohan\n'
         'https://www.instagram.com/qusdygkssla/\n',
 '흥행지수': 17851}
{'다른 이름': 'sic',
 '랭킹': '14',
 '배우 이름': '음문석',
 '생년월일': '1982-12-07',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'],
 '흥행지수': 15263}
{'다른 이름': '전이다',
 '랭킹': '21',
 '배우 이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행지수': 12619}
{'랭킹': '28',
 '배우 이름': '허성태',
 '생년월일': '1977-10-20',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['헌트', '스텔라', '블랙머니', '말모이'],
 '흥행지수': 10308}
{'랭킹': '35',
 '배우 이름': '정만식',
 '생년월일': '1974-12-11',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다'],


In [40]:
'''
5 편의 영화에 출연한 배우만 검색하기 : $size
배우 이름, 출연영화
'''

documents = \
  collection_cine.find({"출연영화":{"$size":5}}, {"배우 이름":1,"출연영화":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    

{'배우 이름': '변요한',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아']}
{'배우 이름': '전혜진', '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사']}
{'배우 이름': '이장원',
 '출연영화': ['뽀로로 극장판 드래곤캐슬 대모험', '후아유', '뽀로로 극장판 보물섬 대모험', '날씨의 아이', '세친구']}


### elemMatch
- 각 document 의 key의 값으로 저장되어 있는 list에
- 적어도 한 개 이상의 요소(item)가 여러 개의 조건을 
- 동시에 만족하는 경우


In [41]:
documents = collection_cine.find({}).limit(3)
for document in documents:
    pprint.pprint(document)

{'_id': ObjectId('6322e0a859013109670e4c40'),
 '랭킹': '7',
 '배우 이름': '변요한',
 '생년월일': '1986-04-29',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '홈페이지': '\n'
         'https://www.facebook.com/saram.byunyohan\n'
         'https://www.instagram.com/qusdygkssla/\n',
 '흥행지수': 17851}
{'_id': ObjectId('6322e0a859013109670e4c41'),
 '다른 이름': 'sic',
 '랭킹': '14',
 '배우 이름': '음문석',
 '생년월일': '1982-12-07',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '육사오(6/45)', '파이프라인'],
 '흥행지수': 15263}
{'_id': ObjectId('6322e0a859013109670e4c42'),
 '다른 이름': '전이다',
 '랭킹': '21',
 '배우 이름': '전혜진',
 '생년월일': '1976-08-10',
 '성별': '여',
 '소속사': '호두엔터테인먼트',
 '신장/체중': '171cm, 49kg',
 '직업': '배우',
 '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사'],
 '취미': '영화감상, 노래 부르기',
 '특기': '수영',
 '학교': '상명대학교 영화과',
 '흥행지수': 12619}


In [42]:
'''
직업이 가수인 배우 중에서 
흥행지수가 가장 높은 순서로 (내림차순) 정렬해서
배우 이름, 직업, 흥행 지수를 조회하세요
'''
documents = collection_cine.find({"직업":"가수"},\
                                 {"배우 이름":1, "직업":1, "흥행지수":1, "_id":0})\
                           .sort("흥행지수", pymongo.DESCENDING) 
for document in documents:
    pprint.pprint(document)

{'배우 이름': '수영', '직업': '가수', '흥행지수': 191}
